## Install - restart kernel afterwards

In [9]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [10]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


## Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium as folium # map rendering library
from folium import plugins
print('Libraries imported.')

Libraries imported.


## Foursquare API

In [2]:
# Define Foursquare Credentials and Version
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [3]:
# Function to obtain all venues for neighborhoods:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)     
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'id',       
                  'Venue Category']
    return(nearby_venues)

In [4]:
# geographical coordinates of Manhattan
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EGXMGBXCIMZJN3HKPGYFSC2IW3KT0JGU0VUOG2GQASXE02VZ&client_secret=WWSHNZUA2TSU1ANJIGGSSOZGYBFDLFR24TOAWN0MM1JKIAYF&v=20180605&ll=40.7896239,-73.9598939&radius=500&limit=100'

# Preparing Datasets

## New York dataset

In [50]:
# Get the dataset
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [51]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [52]:
# loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name'] 
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [53]:
# Use geopy library to get the latitude and longitude data of New York
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
locationNY = geolocator.geocode(address)
latitudeNY = locationNY.latitude
longitudeNY = locationNY.longitude

In [54]:
# Create dataframe venues
venuesNY = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
venuesNY.head()

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,id,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,4c537892fd2ea593cb077a28,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,4d6af9426107f04dedeb297a,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,4c783cef3badb1f7e4244b54,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,508af256e4b0578944c87392,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,4c81a91c51ada1cd87741510,Gas Station


In [57]:
# Analyze Neighborhood
# one hot encoding
NY_onehot = pd.get_dummies(venuesNY[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = venuesNY['Neighborhood']
NY_onehot['Neighborhood_Latitude'] = venuesNY['Neighborhood Latitude'] 
NY_onehot['Neighborhood_Longitude'] = venuesNY['Neighborhood Longitude'] 
NY_onehot['Venue'] = venuesNY['Venue']
NY_onehot['Venue_Latitude'] = venuesNY['Venue Latitude']
NY_onehot['Venue_Longitude'] = venuesNY['Venue Longitude']
NY_onehot['Venue_id'] = venuesNY['id']
NY_onehot['Venue_Category'] = venuesNY['Venue Category']

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]
NY_onehot.head()

,Venue_Category,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical

### Spanish restaurants in New York

In [58]:
restaurantsNY = NY_onehot[['Neighborhood','Spanish Restaurant','Tapas Restaurant','Neighborhood_Latitude','Neighborhood_Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category','Venue_id',]]
# drop all non-Spanish restaurants
a=restaurantsNY.index[restaurantsNY["Spanish Restaurant"] == 0].tolist()
restaurantsNY=restaurantsNY.drop(restaurantsNY.index[a])
# reset index
restaurantsNY.reset_index(inplace=True)
restaurantsNY=restaurantsNY.drop('index', axis=True)
restaurantsNY=restaurantsNY.drop('Spanish Restaurant', axis=True)
restaurantsNY=restaurantsNY.drop('Tapas Restaurant', axis=True)
restaurantsNY

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_id
0,Kingsbridge,40.881687,-73.902818,El Economico Restaurant,40.879330,-73.904597,Spanish Restaurant,4ca785a597c8a1cd7e577ba5
1,Kingsbridge,40.881687,-73.902818,Q'Kachapa,40.880006,-73.904157,Spanish Restaurant,50ea5611e4b044bdeead3ade
2,Norwood,40.877224,-73.879391,El Presidente II,40.880516,-73.883233,Spanish Restaurant,4cdadbde958f236a337da903
3,Baychester,40.866858,-73.835798,Caridad & Louie,40.865843,-73.837707,Spanish Restaurant,4be2ecf6b02ec9b6a5064ec0
4,City Island,40.847247,-73.786488,Rays Cafe Resturant,40.843895,-73.785322,Spanish Restaurant,531f7ea1498e56210466e0bd
5,Bedford Park,40.870185,-73.885512,La Rola Resturant,40.866216,-73.886133,Spanish Restaurant,519aae54498ec1fda8c8ae81
6,Bedford Park,40.870185,-73.885512,Pollo Sabroso,40.869333,-73.890090,Spanish Restaurant,4d5dc3eb29ef236a77b48459
7,University Heights,40.855727,-73.910416,Cuchara Restuarant And Lounge,40.854449,-73.915804,Spanish Restaurant,5a418a6dc666666f8906d1df
8,Morris Heights,40.847898,-73.919672,Yoly Restaurant,40.846042,-73.919716,Spanish Restaurant,4e39863dae60f589a5dc0dcb
9,Fordham,40.860997,-73.896427,Kasike Mofongo House,40.859929,-73.893177,Spanish Restaurant,4cf9d4d12d80a14383df40d8


### Tapas restaurants in New York

In [59]:
tapasNY = NY_onehot[['Neighborhood','Spanish Restaurant','Tapas Restaurant','Neighborhood_Latitude','Neighborhood_Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category','Venue_id',]]
# drop all non-Tapas restaurants
a=tapasNY.index[tapasNY["Tapas Restaurant"] == 0].tolist()
tapasNY=tapasNY.drop(tapasNY.index[a])
# reset index
tapasNY.reset_index(inplace=True)
tapasNY=tapasNY.drop('index', axis=True)
tapasNY=tapasNY.drop('Spanish Restaurant', axis=True)
tapasNY=tapasNY.drop('Tapas Restaurant', axis=True)
tapasNY

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_id
0,Williamsburg,40.707144,-73.958115,Traif,40.710557,-73.958895,Tapas Restaurant,4bb92b3f7421a593b492c240
1,Fort Greene,40.688527,-73.972906,Olea,40.687716,-73.970594,Tapas Restaurant,44fc6a24f964a52084381fe3
2,South Side,40.710861,-73.958001,Traif,40.710557,-73.958895,Tapas Restaurant,4bb92b3f7421a593b492c240
3,Washington Heights,40.851903,-73.936900,Las Tapas,40.855234,-73.937284,Tapas Restaurant,54405a3e498e5d4a84ee266c
4,Washington Heights,40.851903,-73.936900,Lucky 7 Tapas Bar,40.850447,-73.933030,Tapas Restaurant,4e8d32ebb803ca3dfcbd112e
5,Central Harlem,40.815976,-73.943211,Tio Luca Cocina Bistro Latino,40.811649,-73.942832,Tapas Restaurant,51b3f56c7dd23d8b83b48211
6,Chelsea,40.744035,-74.003116,Salinas,40.743667,-74.002932,Tapas Restaurant,4de00d3c1850ff3700a450b0
7,Chelsea,40.744035,-74.003116,Tia Pol,40.747244,-74.004684,Tapas Restaurant,41098f80f964a5209e0b1fe3
8,Greenwich Village,40.726933,-73.999914,Boqueria,40.724800,-74.002220,Tapas Restaurant,49d4eeaff964a520555c1fe3
9,East Village,40.727847,-73.982226,Yuca Bar & Restaurant,40.726055,-73.983633,Tapas Restaurant,41a12c00f964a5203c1e1fe3


### New York Map with Spanish and Tapas Restaurants

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork

In [60]:
# create map of Manhattan using latitude and longitude values
map_newyork = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=10)
# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)
    
for lat, lng, label in zip(tapasNY['Venue_Latitude'], tapasNY['Venue_Longitude'], tapasNY['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)

for lat, lng, label in zip(restaurantsNY['Venue_Latitude'], restaurantsNY['Venue_Longitude'], restaurantsNY['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)

map_newyork

# Manhattan dataset

In [61]:
# cluster only the neighborhoods of Manhattan
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [62]:
# geographical coordinates of Manhattan
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [63]:
# Create dataframe venues
venuesMH = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [64]:
venuesMH.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,id,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,4b4429abf964a52037f225e3,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,4baf59e8f964a520a6f93be3,Yoga Studio


In [65]:
# Analyze Neighborhood
# one hot encoding
manhattan_onehot = pd.get_dummies(venuesMH[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = venuesMH['Neighborhood']
manhattan_onehot['Neighborhood_Latitude'] = venuesMH['Neighborhood Latitude'] 
manhattan_onehot['Neighborhood_Longitude'] = venuesMH['Neighborhood Longitude'] 
manhattan_onehot['Venue'] = venuesMH['Venue']
manhattan_onehot['Venue_Latitude'] = venuesMH['Venue Latitude']
manhattan_onehot['Venue_Longitude'] = venuesMH['Venue Longitude']
manhattan_onehot['Venue_id'] = venuesMH['id']
manhattan_onehot['Venue_Category'] = venuesMH['Venue Category']

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

,Venue_Category,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,

### Spanish restaurants in Manhattan

In [66]:
restaurantsMH = manhattan_onehot[['Neighborhood','Spanish Restaurant','Tapas Restaurant','Neighborhood_Latitude','Neighborhood_Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category','Venue_id',]]
# drop all non-Spanish restaurants
a=restaurantsMH.index[restaurantsMH["Spanish Restaurant"] == 0].tolist()
restaurantsMH=restaurantsMH.drop(restaurantsMH.index[a])
# reset index
restaurantsMH.reset_index(inplace=True)
restaurantsMH=restaurantsMH.drop('index', axis=True)
restaurantsMH=restaurantsMH.drop('Spanish Restaurant', axis=True)
restaurantsMH=restaurantsMH.drop('Tapas Restaurant', axis=True)
restaurantsMH

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_id
0,Chinatown,40.715618,-73.994279,Cervo's,40.714763,-73.991455,Spanish Restaurant,59480a78f62e090a3b4d6eb7
1,Washington Heights,40.851903,-73.936900,Punto de Sabor Restaurant,40.850740,-73.932470,Spanish Restaurant,4bb8cdb053649c74465847fb
2,Washington Heights,40.851903,-73.936900,Mambi Steak House Bar & Grill,40.847615,-73.938143,Spanish Restaurant,4acce300f964a520e2c920e3
3,Inwood,40.867684,-73.921210,La Reina Del Chicharron,40.866357,-73.925012,Spanish Restaurant,4c043cd3f423a593955dd116
4,Inwood,40.867684,-73.921210,La Nueva España,40.867350,-73.921032,Spanish Restaurant,4becab8e0ddaa59385ec1dc3
5,Hamilton Heights,40.823604,-73.949688,Caridad Restaurant,40.826307,-73.950852,Spanish Restaurant,4a2edc71f964a5206f981fe3
6,Manhattanville,40.816934,-73.957385,Mofongo del Valle,40.820317,-73.954977,Spanish Restaurant,4b55274bf964a52006dd27e3
7,East Harlem,40.792249,-73.944182,El Caribeno,40.791550,-73.946723,Spanish Restaurant,4bc65c9cdb8fa593be479c37
8,Lenox Hill,40.768113,-73.958860,Boqueria,40.771510,-73.956073,Spanish Restaurant,542a0b46498e786c48692618
9,Tribeca,40.721522,-74.010683,Tauro Restaurant,40.725336,-74.007591,Spanish Restaurant,4aa9314bf964a520645220e3


### Tapas restaurants in Manhattan

In [67]:
tapasMH = manhattan_onehot[['Neighborhood','Spanish Restaurant','Tapas Restaurant','Neighborhood_Latitude','Neighborhood_Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category','Venue_id',]]
# drop all non-Tapas restaurants
a=tapasMH.index[tapasMH["Tapas Restaurant"] == 0].tolist()
tapasMH=tapasMH.drop(tapasMH.index[a])
# reset index
tapasMH.reset_index(inplace=True)
tapasMH=tapasMH.drop('index', axis=True)
tapasMH=tapasMH.drop('Spanish Restaurant', axis=True)
tapasMH=tapasMH.drop('Tapas Restaurant', axis=True)
tapasMH

,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_id
0,Washington Heights,40.851903,-73.936900,Las Tapas,40.855234,-73.937284,Tapas Restaurant,54405a3e498e5d4a84ee266c
1,Washington Heights,40.851903,-73.936900,Lucky 7 Tapas Bar,40.850447,-73.933030,Tapas Restaurant,4e8d32ebb803ca3dfcbd112e
2,Central Harlem,40.815976,-73.943211,Tio Luca Cocina Bistro Latino,40.811649,-73.942832,Tapas Restaurant,51b3f56c7dd23d8b83b48211
3,Chelsea,40.744035,-74.003116,Salinas,40.743667,-74.002932,Tapas Restaurant,4de00d3c1850ff3700a450b0
4,Chelsea,40.744035,-74.003116,Tia Pol,40.747244,-74.004684,Tapas Restaurant,41098f80f964a5209e0b1fe3
5,Greenwich Village,40.726933,-73.999914,Boqueria,40.724800,-74.002220,Tapas Restaurant,49d4eeaff964a520555c1fe3
6,East Village,40.727847,-73.982226,Yuca Bar & Restaurant,40.726055,-73.983633,Tapas Restaurant,41a12c00f964a5203c1e1fe3
7,Soho,40.722184,-74.000657,Boqueria,40.724800,-74.002220,Tapas Restaurant,49d4eeaff964a520555c1fe3
8,Flatiron,40.739673,-73.990947,Boqueria,40.740200,-73.993770,Tapas Restaurant,44f4b0ebf964a52036381fe3
9,Hudson Yards,40.756658,-74.000111,La Barra,40.752849,-74.001750,Tapas Restaurant,5c9c05744940bc0039e4743d


### Manhattan Map with Spanish and Tapas Restaurants

In [68]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)
    
for lat, lng, label in zip(tapasMH['Venue_Latitude'], tapasMH['Venue_Longitude'], tapasMH['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)

for lat, lng, label in zip(restaurantsMH['Venue_Latitude'], restaurantsMH['Venue_Longitude'], restaurantsMH['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)   

map_manhattan

## Toronto dataset

In [6]:
# get HTML file and scraping with BeautifulSoup
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [7]:
# Create dataframe
# Assign values to lists
A=[]
B=[]
C=[]

for row in My_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
# Create dataframe
import pandas as pd
dfTO=pd.DataFrame()
dfTO['PostalCode']=A
dfTO['Borough']=B
dfTO['Neighborhood']=C

In [8]:
# drop rows with 'Not assigned' value in Borough column
dfTO=dfTO[(dfTO['Borough']!= 'Not assigned')]

# Join columns of same PostalCode and Borough
dfTO=dfTO.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ', '.join(x))
dfTO.reset_index(inplace = True)

# replace Not assigned Neighborhoods with Borough
dfTO['Neighborhood']=dfTO['Neighborhood'].replace('Not assigned\n', dfTO['Borough'])

# sort dataframe by Postalcode
dfTO.sort_values(by='PostalCode', ascending=True, axis=0, inplace=True)

In [9]:
# read in Geospatial dataset
geo = pd.read_csv("http://cocl.us/Geospatial_data")

dfTO['Latitude']=geo['Latitude']
dfTO['Longitude']=geo['Longitude']

In [10]:
# Create dataframe venues
venuesTO = getNearbyVenues(names=dfTO['Neighborhood'],
                                   latitudes=dfTO['Latitude'],
                                   longitudes=dfTO['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood
, Morningside, West Hill
Woburn
Cedarbrae

Scarborough Village
East Birchmount Park
, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West

Birch Cliff, Cliffside West

Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners
, Sullivan
, Tam O'Shanter
Agincourt North, L'Amoreaux East
, Milliken, Steeles East

L'Amoreaux West

Upper Rouge
Hillcrest Village
Fairview
, Henry Farm, Oriole

Bayview Village
Silver Hills
, York Mills
Newtonbrook, Willowdale
Willowdale South

York Mills West

Willowdale West
Parkwoods
Don Mills North

Flemingdon Park, Don Mills South

Bathurst Manor, Downsview North
, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East

Downsview West
Downsview Central

Downsview Northwest

Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toront

In [11]:
venuesTO.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,id,Venue Category
0,"Rouge, Malvern",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,4b6a37ccf964a520a5cd2be3,Department Store
1,"Rouge, Malvern",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,4c6aa500c946e21ec432ed8e,Coffee Shop


In [12]:
# Analyze Neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(venuesTO[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venuesTO['Neighborhood']
toronto_onehot['Neighborhood_Latitude'] = venuesTO['Neighborhood Latitude'] 
toronto_onehot['Neighborhood_Longitude'] = venuesTO['Neighborhood Longitude'] 
toronto_onehot['Venue'] = venuesTO['Venue']
toronto_onehot['Venue_Latitude'] = venuesTO['Venue Latitude']
toronto_onehot['Venue_Longitude'] = venuesTO['Venue Longitude']
toronto_onehot['Venue_id'] = venuesTO['id']
toronto_onehot['Venue_Category'] = venuesTO['Venue Category']

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
#toronto_onehot.head()

In [13]:
restaurantsTO = toronto_onehot[['Neighborhood','Tapas Restaurant', 'Neighborhood_Latitude','Neighborhood_Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category','Venue_id',]]

In [14]:
# drop all non-Tapas restaurants
a=restaurantsTO.index[restaurantsTO["Tapas Restaurant"] == 0].tolist()
restaurantsTO=restaurantsTO.drop(restaurantsTO.index[a])
# reset index
restaurantsTO.reset_index(inplace=True)

In [15]:
restaurantsTO.head()

,index,Neighborhood,Tapas Restaurant,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_id
0,2236,"Kingsview Village, Martin Grove Gardens\n, Ric...",1,43.647927,-79.41975,Carmen,43.644829,-79.415872,Tapas Restaurant,5154f168e4b0d1b01edb977e


### Toronto Map with Spanish and Tapas Restaurants

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.7, -79.3], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(dfTO['Latitude'], dfTO['Longitude'], dfTO['Borough'], dfTO['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
for lat, lng, label in zip(restaurantsTO['Venue_Latitude'], restaurantsTO['Venue_Longitude'], restaurantsTO['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Explore specific venue

In [17]:
venue_id = '5154f168e4b0d1b01edb977e' # ID of Carmen Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5154f168e4b0d1b01edb977e?client_id=EGXMGBXCIMZJN3HKPGYFSC2IW3KT0JGU0VUOG2GQASXE02VZ&client_secret=WWSHNZUA2TSU1ANJIGGSSOZGYBFDLFR24TOAWN0MM1JKIAYF&v=20180605'

In [36]:
results = requests.get(url).json()


In [40]:
# assign relevant part of JSON to venues
venues = results['response']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,venue.id,venue.name,venue.contact.phone,venue.contact.formattedPhone,venue.contact.twitter,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.canonicalUrl,venue.categories,venue.verified,venue.stats.tipCount,venue.url,venue.price.tier,venue.price.message,venue.price.currency,venue.likes.count,venue.likes.groups,venue.likes.summary,venue.dislike,venue.ok,venue.rating,venue.ratingColor,venue.ratingSignals,venue.allowMenuUrlEdit,venue.beenHere.count,venue.beenHere.unconfirmedCount,venue.beenHere.marked,venue.beenHere.lastCheckinExpiredAt,venue.specials.count,venue.specials.items,venue.photos.count,venue.photos.groups,venue.photos.summary,venue.reasons.count,venue.reasons.items,venue.hereNow.count,venue.hereNow.summary,venue.hereNow.groups,venue.createdAt,venue.tips.count,venue.tips.groups,venue.shortUrl,venue.timeZone,venue.listed.count,venue.listed.groups,venue.popular.isOpen,venue.popular.isLocalHoliday,venue.popular.timeframes,venue.pageUpdates.count,venue.pageUpdates.items,venue.inbox.count,venue.inbox.items,venue.attributes.groups,venue.bestPhoto.id,venue.bestPhoto.createdAt,venue.bestPhoto.source.name,venue.bestPhoto.source.url,venue.bestPhoto.prefix,venue.bestPhoto.suffix,venue.bestPhoto.width,venue.bestPhoto.height,venue.bestPhoto.visibility,venue.colors.highlightColor.photoId,venue.colors.highlightColor.value,venue.colors.highlightTextColor.photoId,venue.colors.highlightTextColor.value,venue.colors.algoVersion
0,5154f168e4b0d1b01edb977e,Carmen,4165350404,(416) 535-0404,carmensayz,922 Queen St. W,at Shaw St.,43.644829,-79.415872,"[{'label': 'display', 'lat': 43.64482933866147...",M5T 2L9,CA,Toronto,ON,Canada,"[922 Queen St. W (at Shaw St.), Toronto ON M5T...",https://foursquare.com/v/carmen/5154f168e4b0d1...,"[{'id': '4bf58dd8d48988d1db931735', 'name': 'T...",False,43,http://www.carmensayz.com,3,Expensive,$,69,"[{'type': 'others', 'count': 69, 'items': []}]",69 Likes,False,False,7.9,C5DE35,116,True,0,0,False,0,0,[],148,"[{'type': 'checkin', 'name': 'Friends' check-i...",0 photos,1,"[{'summary': 'Lots of people like this place',...",0,Nobody here,[],1364521320,43,"[{'type': 'others', 'name': 'All tips', 'count...",http://4sq.com/ZER3LZ,America/Toronto,96,"[{'type': 'others', 'name': 'Lists from other ...",False,False,"[{'days': 'Today', 'includesToday': True, 'ope...",0,[],0,[],"[{'type': 'price', 'name': 'Price', 'summary':...",53946abd498e7c19d0059006,1402235581,Foursquare for iOS,https://foursquare.com/download/#/iphone,https://fastly.4sqi.net/img/general/,/37420158_PDYSGSf7JagGlcb_7VuA0kXEPB_78uhXpEAa...,720,960,public,53946abd498e7c19d0059006,-9433080,53946abd498e7c19d0059006,-1,3


In [47]:
print(dataframe['venue.price.message'])

0    Expensive
Name: venue.price.message, dtype: object


In [46]:
print(dataframe['venue.rating'])

0    7.9
Name: venue.rating, dtype: float64


In [49]:
print(dataframe['venue.reasons.items'])

0    [{'summary': 'Lots of people like this place',...
Name: venue.reasons.items, dtype: object
